In [1]:
import cv2,os,time
os.sys.path.append('../utilities/')
from datetime import datetime
import numpy as np
import IPython.display as Disp
import support_utility_openvino
import matplotlib.pyplot as plt
import ipywidgets as widgets
from shapely.geometry import Polygon,Point


In [2]:
bag_detector = support_utility_openvino.async_infer()


Available Devices :  ['CPU', 'GNA', 'GPU']


In [4]:
bag_detector.load_model("/home/prateek/shared_space/DATA_N_MODELS/useful_models/bag_counter/v4/openvino_model/frozen_inference_graph.xml","CPU")

OPTIMIZATION_CAPABILITIES for CPU:  ['FP32', 'FP16', 'INT8', 'BIN']
model inputs : dict_keys(['image_tensor'])
model outputs :  dict_keys(['DetectionOutput'])


In [7]:
output_support = support_utility_openvino.create_plot()

In [8]:
from tracker_2 import CentroidTracker
ct= CentroidTracker(maxDistance=300,minAppeared=5, maxDisappeared=10)

In [9]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

In [14]:
channel = "/media/prateek/shared_space/DATA_N_MODELS/useful_vids/bag_Counter/2021_01_25/TL1_Noon.mp4"
cap = cv2.VideoCapture(channel)
cap.set(1,3000)
cv2.namedWindow("preview",cv2.WINDOW_NORMAL)
bag_detector.reinit_model()
thresh = .2
counting_line_location = .4
min_appeared = 10
write_video = False
total_count = 0
if write_video:
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter("../output_vids/bag_counter/"+os.path.basename(channel)+"_output.mp4",fourcc, 20, (int(cap.get(3)),int(cap.get(4))))
    
while True:
    t0 = time.time()
    ret,frame = cap.read()
#     frame = rotate_image(frame,-10)
    bag_detector.predict(frame)
    if bag_detector.frame_processed>=bag_detector.num_requests:
        frame,attr,res = bag_detector.postprocess_op()
        bags,bboxes = output_support.trim_frame_with_result(frame,res[0],threshold=thresh)
        objects = ct.update(bboxes)

        # loop over the tracked objects
        for idx,(objectID, centroid) in enumerate(objects.items()):
            appeared_frames = ct.appeared[objectID] 
            incremented = False
            if not ct.count_dict[objectID]:
                if centroid[0] > frame.shape[1]*counting_line_location and appeared_frames>min_appeared:
                    total_count+=1
                    ct.count_dict[objectID] = True
                    incremented = True
                # draw both the ID of the object and the centroid of the
            # object on the output frame
            text = "ID {}".format(objectID)
            frame_op = cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            frame_op = cv2.circle(frame_op, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

        frame_op = output_support.label_obj_detection(frame,res[0],labels=["bags"],thresh=thresh)
        t1 = time.time()
        frame_op = output_support.write_text(frame_op,"FPS= %.4s"%(1/(t1-t0)),text_color=(0,0,0))
        frame_op = output_support.write_text(frame_op,"Count = %s"%total_count,text_color=(0,0,255),location="top-right")

        if write_video:
            out.write(frame_op)
        cv2.imshow("preview",cv2.resize(frame_op,(1920,1080)))
        k = cv2.waitKey(1)
        if k==ord('q'):
            break
if write_video:
    out.release()
cv2.destroyAllWindows()
cap.release()
        

        
    
    

Internals of neural network is not easy to understand, but to create an autonomous system, multiple of such neural network have to be combined. 
What's the best way to fine tune such large meta/multi task learning?